# Import Dependences

In [1]:
import os
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 https://develo

In [2]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-29 20:00:10--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-11-29 20:00:11 (10.5 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Establish a Spark session and add the Postgres driver to the filepath
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AMZreviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract Data

In [4]:
# Read the first TSV.GZ file from an S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
books_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
books_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [5]:
# Count the amount of rows in the dataset
print(f"Rows in dataset: {books_df.count()}")

Rows in dataset: 3105520


# Transform Data

In [6]:
# Count the amount of distinct rows in the dataset to find duplicates
print(f"Distinct rows in dataset: {books_df.distinct().count()}")

Distinct rows in dataset: 3105520


In [7]:
# Drop unnecessary columns
books_df = books_df.drop("marketplace", "product_category", "verified_purchase", "review_headline", "review_body")

books_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|          4|            2|          3|   N| 2005-10-14|
|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|          3|            5|          5|   N| 2005-10-14|
|   12257412|R1DOSHH6AI622S|1844161560|     253182049|Eisenhorn (A Warh...|          4|            1|         22|   N| 2005-10-14|
|   50732546| RATOTLA3OF70O|0373836635|     348672532|Colby Conspiracy ...|          5|            2|          2|   N| 2005-10-14|
|   51964897|R1TNWRKIVHVYOV|0262181533|     598678717|The Psychology of...|        

In [8]:
# Find unique customer_id counts
print(f"Unique customer_id counts: {books_df.select('customer_id').distinct().count()}")

Unique customer_id counts: 1502380


In [9]:
# Find instances of each customer_id
customer_id_count_df = books_df.groupBy('customer_id').count()

books_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|          4|            2|          3|   N| 2005-10-14|
|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|          3|            5|          5|   N| 2005-10-14|
|   12257412|R1DOSHH6AI622S|1844161560|     253182049|Eisenhorn (A Warh...|          4|            1|         22|   N| 2005-10-14|
|   50732546| RATOTLA3OF70O|0373836635|     348672532|Colby Conspiracy ...|          5|            2|          2|   N| 2005-10-14|
|   51964897|R1TNWRKIVHVYOV|0262181533|     598678717|The Psychology of...|        

In [10]:
# Join count to original dataframe
books_df = books_df.join(customer_id_count_df, books_df.customer_id == customer_id_count_df.customer_id, "left").drop(books_df.customer_id)
books_df.show()

+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+-----+
|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|customer_id|count|
+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+-----+
|R1DOSHH6AI622S|1844161560|     253182049|Eisenhorn (A Warh...|          4|            1|         22|   N| 2005-10-14|   12257412|    1|
|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|          3|            5|          5|   N| 2005-10-14|   12703090|    1|
|R2TIO43ND87XVO|0375701907|     635714194|Straight Man: A N...|          5|            1|          1|   N| 2005-10-14|   15437121|    1|
| R7M06Z88PD7SX|0029148510|      72387289|Acts of War: Beha...|          4|           14|         14|   N| 2005-10-14|   27925116|    9|
|R2F53LI9KK9MOY|0316769487|     862964341

In [11]:
# Ensure rows are preserved
print(books_df.count())

3105520


In [12]:
# Rename count column
books_df = books_df.withColumnRenamed("count", "customer_count")
books_df.show()

+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+--------------+
|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|customer_id|customer_count|
+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+--------------+
|R1DOSHH6AI622S|1844161560|     253182049|Eisenhorn (A Warh...|          4|            1|         22|   N| 2005-10-14|   12257412|             1|
|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|          3|            5|          5|   N| 2005-10-14|   12703090|             1|
|R2TIO43ND87XVO|0375701907|     635714194|Straight Man: A N...|          5|            1|          1|   N| 2005-10-14|   15437121|             1|
| R7M06Z88PD7SX|0029148510|      72387289|Acts of War: Beha...|          4|           14|         14|   N| 2005-10-14|   279

In [13]:
# Check data types
books_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = true)



In [14]:
# Cast customer_count to integer
books_df = books_df.withColumn("customer_count", books_df.customer_count.cast("integer"))
books_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = true)



In [15]:
# Create review_id dataframe for review_id_table in our database
review_id_df = books_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_df.show() 

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1DOSHH6AI622S|   12257412|1844161560|     253182049| 2005-10-14|
|  RF6IUKMGL8SF|   12703090|0811828964|      56191234| 2005-10-14|
|R2TIO43ND87XVO|   15437121|0375701907|     635714194| 2005-10-14|
| R7M06Z88PD7SX|   27925116|0029148510|      72387289| 2005-10-14|
|R2F53LI9KK9MOY|   31048862|0316769487|     862964341| 2005-10-14|
|R3SJYLG07EHNE9|   45232107|013146308X|     332774292| 2005-10-14|
|R1VE0FQQ0QTQJN|   48541186|0300108834|     915891133| 2005-10-14|
| RATOTLA3OF70O|   50732546|0373836635|     348672532| 2005-10-14|
|R17026W0TBFZGL|   50732546|045121692X|     904844135| 2005-10-14|
|R18TY4WD19OUAO|   52157117|141378240X|     656010659| 2005-10-14|
|R21SYDQ70ILUC0|   52253037|1580085695|     586052746| 2005-10-14|
|R2YDYRSLGNHPHR|   52534781|0811848833|     102333549| 2005-10

In [16]:
# Create products dataframe for products in our database
products_df = books_df.select(['product_id', 'product_title'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1844161560|Eisenhorn (A Warh...|
|0811828964|The Bad Girl's Gu...|
|0375701907|Straight Man: A N...|
|0029148510|Acts of War: Beha...|
|0316769487|The Catcher in th...|
|013146308X|The Pterosaurs: F...|
|0300108834|A Little History ...|
|0373836635|Colby Conspiracy ...|
|045121692X|The Last Heiress ...|
|141378240X|   Lipstick's Legacy|
|1580085695|Furry Logic: A Gu...|
|0811848833|The Star Wars Pos...|
|0385730586|Sisterhood of the...|
|1592285570|Temple to the Win...|
|0446531081|Hour Game (King &...|
|0976822105|     Faith is a Verb|
|0782144276|Photoshop for Nat...|
|1581603681|Opening Combinati...|
|0375757996|The Basic Works o...|
|0262181533|The Psychology of...|
+----------+--------------------+
only showing top 20 rows



In [17]:
# Create customers dataframe for customers in our database
customers_df = books_df.select(['customer_id', 'customer_count'])
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12257412|             1|
|   12703090|             1|
|   15437121|             1|
|   27925116|             9|
|   31048862|             2|
|   45232107|             1|
|   48541186|             1|
|   50732546|          9964|
|   50732546|          9964|
|   52157117|             2|
|   52253037|             1|
|   52534781|             1|
|   12076615|             1|
|   15005044|             1|
|   15280864|             1|
|   24209213|             1|
|   26217071|             1|
|   29732693|             1|
|   37684582|             1|
|   51964897|             8|
+-----------+--------------+
only showing top 20 rows



In [18]:
# Create vine dataframe for vine_table in our database
vine_df = books_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1DOSHH6AI622S|          4|            1|         22|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R2TIO43ND87XVO|          5|            1|          1|   N|
| R7M06Z88PD7SX|          4|           14|         14|   N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R17026W0TBFZGL|          5|            1|          2|   N|
|R18TY4WD19OUAO|          5|            1|          1|   N|
|R21SYDQ70ILUC0|          5|            1|          1|   N|
|R2YDYRSLGNHPHR|          4|           16|         19|   N|
| RQ58W7SMO911M|          4|            2|          3|   N|
| RRS38KZ4WB5O2|          5|            

# Load Data

In [19]:
# Fill out aws fields
my_aws_endpoint = 'amazon-big-data-db.cbsmtzrxs98f.us-east-2.rds.amazonaws.com'
my_aws_port_number = '5432'
my_aws_database_name = 'amazon_big_data_db'
my_aws_username = 'herzf012'
my_aws_password = 'Lief!997'

In [20]:
# Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

# Choose to append the data
mode = 'append'

In [ ]:
# Write the dataframe to the appropriate table in our PostgreSQL RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)